In [1]:
import pandas as pd
import numpy as np

In [2]:
housing_data = pd.read_csv("data/housing.csv")
housing_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
del housing_data["ocean_proximity"]

## Question 1

In [4]:
# Find a feature with missing values. How many missing values does it have?
housing_data.isnull().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

## Question 2
What's the median (50% percentile) for variable 'population'?

In [5]:
housing_data["population"].describe()

count    20640.000000
mean      1425.476744
std       1132.462122
min          3.000000
25%        787.000000
50%       1166.000000
75%       1725.000000
max      35682.000000
Name: population, dtype: float64

Split the data

In [6]:
n = len(housing_data)
n_val = int(n*0.2)
n_test = int(n*0.2)
n_train = n - n_val - n_test
n_val, n_test, n_train, n

n = len(housing_data)
idx =np.arange(n)
np.random.seed(42)
np.random.shuffle(idx)

df_train = housing_data.iloc[idx[:n_train]]
df_val = housing_data.iloc[idx[n_train:n_train+n_val]]
df_test = housing_data.iloc[idx[n_train+n_val:]]

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


In [7]:
y_train = np.log1p(df_train.median_house_value.values)
y_val = np.log1p(df_val.median_house_value.values)
y_test = np.log1p(df_test.median_house_value.values)

In [8]:
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

## Question 3
- We need to deal with missing values for the column from Q1.
- We have two options: fill it with 0 or with the mean of this variable.
- Try both options. For each, train a linear regression model without regularization using the code from the lessons.
- For computing the mean, use the training only!
- Use the validation dataset to evaluate the models and compare the RMSE of each option.
- Round the RMSE scores to 2 decimal digits using round(score, 2)
- Which option gives better RMSE?

In [9]:
def prepare_X(df, method):
    '''
    Arguments:
        df: (pd.DataFrame) dataframe for training data
        method: (float) numerical value to fill missing values in our dataset
    Return:
        X: (np.array) numpy matrix that stores training data
    '''
    df_num = df.fillna(method)
    X = df_num.values
    return X

def rmse(y, y_pred):

    se = (y - y_pred) ** 2
    mse = se.mean()

    return np.sqrt(mse)

def train_linear_regression(X, y):
	ones = np.ones(X.shape[0])
	X = np.column_stack([ones, X])
	
	XTX = X.T.dot(X)
	XTX_inv = np.linalg.inv(XTX)
	w_full = XTX_inv.dot(X.T).dot(y)
	return w_full[0], w_full[1:]

In [10]:
# Validate results handle missing values with 0
# training
X_train = prepare_X(df_train, 0)
w0, w = train_linear_regression(X_train, y_train)

# validate
X_val = prepare_X(df_val, 0)
y_pred = w0 + X_val.dot(w)

rmse(y_val, y_pred).round(2)

0.33

In [11]:
# Handle missing values with the mean of the training data
# training
mean_fill = df_train['total_bedrooms'].mean()

X_train = prepare_X(df_train, mean_fill)
w0, w = train_linear_regression(X_train, y_train)

# validate
X_val = prepare_X(df_val, mean_fill)
y_pred = w0 + X_val.dot(w)

rmse(y_val, y_pred).round(2)

0.33

Both are equally good

## Question 4

In [12]:
def train_linear_regression_reg(X, y, r=0.001):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX = XTX + r * np.eye(XTX.shape[0])

    XTX_inv = np.linalg.inv(XTX)
    w_full = XTX_inv.dot(X.T).dot(y)

    return w_full[0], w_full[1:]

In [13]:
# Validate results handle missing values with 0
# training
X_train = prepare_X(df_train, 0)

X_val = prepare_X(df_val, 0)

reg_param = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]

for i in reg_param:
    w0, w = train_linear_regression_reg(X_train, y_train, r=i)
    y_val_reg = w0 + X_val.dot(w)
    print(f'The regularization value of {i} gives the RMSE of {rmse(y_val_reg, y_pred).round(2)}')

The regularization value of 0 gives the RMSE of 0.01
The regularization value of 1e-06 gives the RMSE of 0.01
The regularization value of 0.0001 gives the RMSE of 0.01
The regularization value of 0.001 gives the RMSE of 0.01
The regularization value of 0.01 gives the RMSE of 0.01
The regularization value of 0.1 gives the RMSE of 0.01
The regularization value of 1 gives the RMSE of 0.05
The regularization value of 5 gives the RMSE of 0.08
The regularization value of 10 gives the RMSE of 0.09


The regularization value of 0 gives the smallest RMSE

## Question 5

In [14]:
def linear_regression_pipeline(df, seed):
    n = len(df)
    n_val = int(n*0.2)
    n_test = int(n*0.2)
    n_train = n - n_val - n_test
    idx =np.arange(n)
    np.random.seed(seed)
    np.random.shuffle(idx)

    df_train = df.iloc[idx[:n_train]]
    df_val = df.iloc[idx[n_train:n_train+n_val]]
    df_test = df.iloc[idx[n_train+n_val:]]

    y_train = np.log1p(df_train.median_house_value.values)
    y_val = np.log1p(df_val.median_house_value.values)
    y_test = np.log1p(df_test.median_house_value.values)

    del df_train['median_house_value']
    del df_val['median_house_value']
    del df_test['median_house_value']

    # Validate results handle missing values with 0
    # training
    X_train = prepare_X(df_train, 0)
    w0, w = train_linear_regression(X_train, y_train)

    # validate
    X_val = prepare_X(df_val, 0)
    y_pred = w0 + X_val.dot(w)

    return rmse(y_val, y_pred).round(2)

In [15]:
scores = []
for i in range(0,10):
    scores.append(linear_regression_pipeline(housing_data, i))

In [16]:
np.std(scores).round(3)

0.005

## Question 6
- Split the dataset like previously, use seed 9.
- Combine train and validation datasets.
- Fill the missing values with 0 and train a model with r=0.001.
- What's the RMSE on the test dataset?

In [17]:
n = len(housing_data)
n_val = int(n*0.2)
n_test = int(n*0.2)
n_train = n - n_val - n_test
idx =np.arange(n)
np.random.seed(9)
np.random.shuffle(idx)

df_train = housing_data.iloc[idx[:n_train]]
df_val = housing_data.iloc[idx[n_train:n_train+n_val]]
df_test = housing_data.iloc[idx[n_train+n_val:]]


frames = [df_train, df_val]
df_train_val = pd.concat(frames)

df_train_val = df_train_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train_val_orig = df_train_val.median_house_value.values
y_test_orig = df_test.median_house_value.values

y_train_val = np.log1p(y_train_val_orig)
y_test = np.log1p(y_test_orig)

del df_train_val['median_house_value']
del df_test['median_house_value']

X_null_train_val = prepare_X(df_train_val, 0)
w_0_train_val, w_train_val = train_linear_regression_reg(X_null_train_val, y_train_val, r=0.001)

X_null_test = prepare_X(df_test, 0)
y_null_pred_test = w_0_train_val + X_null_test.dot(w_train_val)

np.round(rmse(y_test, y_null_pred_test),2)

0.35

In [23]:
# in production we send requests to a server to get an observation in the form of dictionary
house = df_test.iloc[20].to_dict()
house

{'longitude': -118.21,
 'latitude': 33.95,
 'housing_median_age': 35.0,
 'total_rooms': 2134.0,
 'total_bedrooms': 650.0,
 'population': 2248.0,
 'households': 587.0,
 'median_income': 2.2988}

In [24]:
# we convert into a dataframe to use our preparation function
df_small = pd.DataFrame([house])
df_small

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
0,-118.21,33.95,35.0,2134.0,650.0,2248.0,587.0,2.2988


In [27]:
X_small = prepare_X(df_small, 0)
X_small

array([[-118.21  ,   33.95  ,   35.    , 2134.    ,  650.    , 2248.    ,
         587.    ,    2.2988]])

In [31]:
# get the prediction
y_pred = w0 + X_small.dot(w)
y_pred = y_pred[0]
y_pred # this is a logaritmic value

11.836464405523659

In [32]:
# we convert into 
np.expm1(y_pred)

138199.99714533627

In [34]:
np.expm1(y_test[20])

153400.00000000006